## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-10-18-08-53-29 +0000,bbc,Israel confirms latest body returned from Gaza...,https://www.bbc.com/news/articles/c5yp3eq9knvo...
1,2025-10-18-08-45-06 +0000,wapo,Not so proud to be American — ‘fed up’ expats ...,https://www.washingtonpost.com/world/2025/10/1...
2,2025-10-18-08-32-02 +0000,nyt,Trump Backs Off Tomahawk Suggestion After Zele...,https://www.nytimes.com/live/2025/10/17/us/tru...
3,2025-10-18-08-25-00 +0000,wsj,Trump’s Middle East Peace Plan Hits an Early S...,https://www.wsj.com/world/middle-east/trumps-m...
4,2025-10-18-08-23-10 +0000,wapo,Want to quit the U.S.A? A guide for renouncing...,https://www.washingtonpost.com/world/2025/10/1...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2350/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
16,trump,53
62,new,18
261,ukraine,15
237,government,12
317,andrew,11


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
60,2025-10-18-00-06-00 +0000,wsj,President Trump has commuted the sentence of f...,https://www.wsj.com/politics/trump-commutes-se...,108
33,2025-10-18-03-30-12 +0000,nypost,Disgraced ex-Long Island Rep. George Santos re...,https://nypost.com/2025/10/17/us-news/george-s...,105
142,2025-10-17-20-49-26 +0000,nypost,"Trump urges Russia, Ukraine to end war ‘where ...",https://nypost.com/2025/10/17/us-news/trump-sa...,102
70,2025-10-17-23-45-00 +0000,wsj,Trump Says He Would Rather End War Than Send T...,https://www.wsj.com/world/trump-says-hed-rathe...,99
66,2025-10-17-23-50-52 +0000,nypost,Salesforce CEO Marc Benioff apologizes for say...,https://nypost.com/2025/10/17/us-news/salesfor...,98


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
60,108,2025-10-18-00-06-00 +0000,wsj,President Trump has commuted the sentence of f...,https://www.wsj.com/politics/trump-commutes-se...
182,71,2025-10-17-18-19-43 +0000,nypost,Jeffrey Epstein-linked Prince Andrew forced to...,https://nypost.com/2025/10/17/world-news/princ...
218,53,2025-10-17-16-42-19 +0000,nypost,John Bolton spotted still using White House ba...,https://nypost.com/2025/10/17/us-news/john-bol...
319,49,2025-10-17-10-00-00 +0000,wapo,"The average new-car price hit a record $50,000...",https://www.washingtonpost.com/business/2025/1...
142,49,2025-10-17-20-49-26 +0000,nypost,"Trump urges Russia, Ukraine to end war ‘where ...",https://nypost.com/2025/10/17/us-news/trump-sa...
158,47,2025-10-17-19-56-00 +0000,wsj,Bank of Canada Gov. Tiff Macklem said data nex...,https://www.wsj.com/articles/bank-of-canada-su...
335,46,2025-10-17-09-04-01 +0000,nyt,What We Know About the Latest ‘No Kings’ Prote...,https://www.nytimes.com/2025/10/17/us/no-kings...
66,45,2025-10-17-23-50-52 +0000,nypost,Salesforce CEO Marc Benioff apologizes for say...,https://nypost.com/2025/10/17/us-news/salesfor...
250,44,2025-10-17-15-18-45 +0000,nyt,"Baek Sehee, South Korean Author of ‘I Want to ...",https://www.nytimes.com/2025/10/17/arts/baek-s...
323,33,2025-10-17-09-59-55 +0000,cbc,Madagascar's military coup leader is sworn in ...,https://www.cbc.ca/news/world/madagascar-milit...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
